# 1. Blocker Fraud Company
 <font size="2"> Dados e problema negócio hipotéticos</font>
 
A Blocker Fraud Company é uma empresa especializada na detecção de fraudes em transações financeiras feitas atravś de dispositivos móveis.

O modelo de negócio da empresa é do tipo Serviço com a monetização feita por performance do serviço prestado, ou seja, o usuário paga uma taxa fixa sobre o sucesso da detecção de fraude das transações.

## 1.1. Estratégia de expansão

A Blocker Fraud Company está em fase de expansão no Brasil e para adquirir clientes mais rapidamente, ela adotou uma estratégia muito agressiva. A estratégia funciona da seguinte forma:

1. A empresa vai receber 25% do valor de cada transação detectada verdadeiramente como fraude.
2. A empresa vai receber 5% do valor de cada transação detectada como fraude, porém a transação é verdadeiramente legítima.
3. A empresa vai devolver 100% do valor para o cliente, a cada transação detectada como legítima, porém a transação é verdadeiramente uma fraude.

Com essa estratégia agressiva a empresa assume os riscos em falhar na detecção de fraude e é remunerada na detecção assertiva das fraudes.

## 1.2. Objetivo

Criar um modelo de alta precisão e acurácia na detecção de fraudes de transações feitas através de dispositivos móveis.

### 1.2.1. Entregáveis

Modelo em produção no qual seu acesso será feito via API, ou seja, os clientes enviarão suas transações via API para que o seu modelo as classifique como fraudulentas ou legítimas.

Além disso, você precisará entregar um relatório reportando a performance e os resultados do seu modelo em relação ao lucro e prejuízo que a empresa terá ao usar o modelo que você produziu. No seu relatório deve conter as respostas para as seguintes perguntas:

1. Qual a Precisão e Acurácia do modelo?
2. Qual a Confiabilidade do modelo em classificar as transações como legítimas ou fraudulentas?
3. Qual o Faturamento Esperado pela Empresa se classificarmos 100% das transações com o modelo?
4. Qual o Prejuízo Esperado pela Empresa em caso de falha do modelo?
5. Qual o Lucro Esperado pela Blocker Fraud Company ao utilizar o modelo?


# 2. Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#display
pd.set_option('display.float_format', lambda x: '%.5f' % x)

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

## 2.1. Utils functions

In [2]:
def create_features(df: pd.DataFrame) -> pd.DataFrame:
    """
        Cria as novas features a partir dos dados de entrada
        Parameters
        ----------
        df: Pandas DataFrame

        Returns
        -------
        df: Pandas DataFrame
            Novo dataframe com as features inseridas.
    """
    df['is_high_amount'] = define_high_amount(df, 179000)
    df['is_M_customer'] = define_merchant_customer(df)
    df['is_new_balance_orig_zero'] =  define_is_balance_zero(df, 'newbalanceOrig')
    df['is_new_balance_dest_zero'] = define_is_balance_zero(df, 'newbalanceDest')
    df['diff_orig'] = difference_between_two_columns(df, 'oldbalanceOrg', 'newbalanceOrig')
    df['diff_dest'] = difference_between_two_columns(df, 'oldbalanceDest', 'newbalanceDest')
    df['is_orig_old_balance_greater_new'] = define_column_is_greater(df, 'oldbalanceOrg','newbalanceOrig')
    df['is_dest_old_balance_greater_new'] = define_column_is_greater(df, 'oldbalanceDest','newbalanceDest')
    return df

def define_high_amount(df: pd.DataFrame, value: int) -> np.array:
    is_high_amount = np.where(df['amount']>=value, 1, 0)
    return is_high_amount

def define_merchant_customer(df: pd.DataFrame) -> np.array:
    is_merchant_customer = np.where(df['nameDest'].str.contains('M'), 1, 0)
    return is_merchant_customer

def define_is_balance_zero(df: pd.DataFrame, column: str) -> np.array:
    is_column_balance_zero = np.where(df[column]==0, 1, 0)
    return is_column_balance_zero

def difference_between_two_columns(df: pd.DataFrame, left_column: str, right_column: str) -> pd.Series:
    difference_result = df[left_column]-df[right_column]
    return difference_result

def define_column_is_greater(df: pd.DataFrame, left_column: str, right_column: str) -> np.array:
    is_greater_result = np.where(df[left_column]>df[right_column], 1, 0)
    return is_greater_result

# 3. Data

## 3.1. Load dataset

In [3]:
df = pd.read_parquet('df_train.parquet.gzip')

In [4]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64000,C1231006815,170136.00000,160296.36000,M1979787155,0.00000,0.00000,0,0
1,1,PAYMENT,1864.28000,C1666544295,21249.00000,19384.72000,M2044282225,0.00000,0.00000,0,0
2,1,TRANSFER,181.00000,C1305486145,181.00000,0.00000,C553264065,0.00000,0.00000,1,0
3,1,CASH_OUT,181.00000,C840083671,181.00000,0.00000,C38997010,21182.00000,0.00000,1,0
4,1,PAYMENT,11668.14000,C2048537720,41554.00000,29885.86000,M1230701703,0.00000,0.00000,0,0


# 4. Modelling Data

A partir do momento que identificamos alguns insights da etapa anterior, análise exploratória, partimos então para a modelagem dos dados atuais para que seja possível a inserção de novas variáveis afim de auxiliar os algoritmos a entenderem o espaço de features e com isso aumentando a sua possibilidade de prever se a transação foi fraudulenta ou não.

## 4.1. Applying utils functions

Foi desenvolvido uma função onde foi reunido algumas manipulações para a criação de novas features. Favor verificar na seção ``` 2.1. Utils functions```  sobre essa função.

In [5]:
df_new = df.copy()

In [6]:
df_applied_features = create_features(df_new)

In [7]:
df_applied_features.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,is_high_amount,is_M_customer,is_new_balance_orig_zero,is_new_balance_dest_zero,diff_orig,diff_dest,is_orig_old_balance_greater_new,is_dest_old_balance_greater_new
0,1,PAYMENT,9839.64000,C1231006815,170136.00000,160296.36000,M1979787155,0.00000,0.00000,0,0,0,1,0,1,9839.64000,0.00000,1,0
1,1,PAYMENT,1864.28000,C1666544295,21249.00000,19384.72000,M2044282225,0.00000,0.00000,0,0,0,1,0,1,1864.28000,0.00000,1,0
2,1,TRANSFER,181.00000,C1305486145,181.00000,0.00000,C553264065,0.00000,0.00000,1,0,0,0,1,1,181.00000,0.00000,1,0
3,1,CASH_OUT,181.00000,C840083671,181.00000,0.00000,C38997010,21182.00000,0.00000,1,0,0,0,1,1,181.00000,21182.00000,1,1
4,1,PAYMENT,11668.14000,C2048537720,41554.00000,29885.86000,M1230701703,0.00000,0.00000,0,0,0,1,0,1,11668.14000,0.00000,1,0


## 4.4. Save new dataset

In [8]:
df_applied_features.to_parquet('df_applied_features.parquet.gzip',
              compression='gzip')

## 4.3. Conclusion

Com os dados novos inseridos com as novas features, se faz necessário o salvar desses dados, para tal, foi escolhido salvar no formato parquet e comprimido para que seja possível a diminuição do tamanho do arquivo final, aqui temos uma diminuição de metade comparada com o arquivo csv anterior.

A partir de agora, esse novo arquivo será o que usaremos para o restante do processo, inicaremos agora a etapa de preparação de dados e a feature engineering.